In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## パッケージの読み込み

In [ ]:
import numpy as np
import pandas as pd

## データの読み込み

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')  
test = pd.read_csv('../input/titanic/test.csv')  
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
gender_submission.head()

In [ ]:
data = pd.concat([train, test], sort=False)

In [ ]:
data.isnull().sum()

## 特徴量エンジニアリング

In [ ]:
# data['Sex'].replace({'male': 0, 'female': 1}, inplace=True) # 将来的に使えなくなる?
data['Sex'] = data['Sex'].replace({'male': 0, 'female': 1}).astype(int)
data.head()

In [ ]:
data['Embarked'] = data['Embarked'].fillna('S')
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data.head()

mapはreplaceと似た処理だが、元のデータに変換を指示していない値が存在した場合、np.nan（欠損値）に置き換わる。

### Fareの欠損値を平均値で置き換える

In [ ]:
data['Fare'] = data['Fare'].fillna(np.mean(data['Fare']))

### 年齢を平均値と標準偏差をもとに乱数で決める

In [ ]:
age_avr = data['Age'].mean()
age_std = data['Age'].std()
data['Age'] = data['Age'].fillna(np.random.randint(age_avr - age_std, age_avr + age_std))
data.head()

### 使用しないカラムを削除

In [ ]:
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data = data.drop(delete_columns, axis=1) # axis=0 -> raw, axis=1 -> column
data.head()

In [ ]:
# インデクシング
train = data[:len(train)]
test = data[len(train):]

In [ ]:
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

# 機械学習アルゴリズムの学習.予測

## ロジスティック回帰

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

clf = Classifier(分類器)、penalty＝損失,L2正則化, sag=Stochastic Average Gradient(確率的平均勾配)

## submit

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission.csv', index=False)

### list(map(A, B))

In [ ]:
# # map関数に引き渡すイテラブル「sample1_list」の定義
#     # range(5)の実行によりsample1_listの中には「0,1,2,3,4」の値が入っている
#     sample1_list = list(range(5))

#     # イテラブルを処理する関数「multi」の定義
#     def multi(x):
#         y = x*2
#         return y

#     # map関数を呼び出し、multi関数にイテラブルsample1_listを引き渡す
#     sample2_list = map(multi, sample1_list)

#     # multi関数からの戻り値sample2_listの値をプリントアウトして確認
#     print(list(sample2_list))

[0, 2, 4, 6, 8]

## Pandas Profiling

In [ ]:
import pandas as pd
import pandas_profiling

train = pd.read_csv('../input/titanic/train.csv')
train.profile_report()

## matplotlib & seabornの使い方

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.hist(train.loc[train['Survived'] == 0, 'Age'].dropna(), # binsはバーの数
        bins=30, alpha=0.5, label='0')
plt.hist(train.loc[train['Survived'] == 1, 'Age'].dropna(),
        bins=30, alpha=0.5, label='1')
plt.xlabel('Age')
plt.ylabel('count')
plt.legend(title='Survived')

In [ ]:
sns.countplot(x='SibSp', hue='Survived', data=train)
plt.legend(loc='upper right', title='Survived')

In [ ]:
sns.countplot(x='Parch', hue='Survived', data=train)
plt.legend(loc='upper right', title='Survived')

In [ ]:
plt.hist(train.loc[train['Survived'] == 0, 'Fare'].dropna(),
        range=(0, 250), bins=25, alpha=0.5, label='0')
plt.hist(train.loc[train['Survived'] == 1, 'Fare'].dropna(),
        range=(0, 250), bins=25, alpha=0.5, label='1')
plt.xlabel('Fare')
plt.ylabel('count')
plt.legend(title='Survived')
plt.xlim(-5, 250)

In [ ]:
sns.countplot(x='Pclass', hue='Survived', data=train)

In [ ]:
sns.countplot(x='Sex', hue='Survived', data=train)

In [ ]:
sns.countplot(x='Embarked', hue='Survived', data=train)

# 特徴量エンジニアリング

### データを加工済みのため、新しく設定し直す

In [ ]:
import numpy as np
import pandas as pd


train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'] = data['Sex'].replace({'male':0, 'female':1}).astype(int)
data['Embarked'] = data['Embarked'].fillna(('S'))
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'] = data['Fare'].fillna(np.mean(data['Fare']))
data['Age'] = data['Age'].fillna(data['Age'].median())
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
X_train.head()

In [ ]:
import seaborn as sns

data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
train['FamilySize'] = data['FamilySize'][:len(train)]
test['FamilySize'] = data['FamilySize'][len(train):]
sns.countplot(x='FamilySize', data=train, hue='Survived')

## 勾配ブースティング

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.predict(X_test)

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission_randomforest.csv', index=False)
sub.head()

## ホールドアウト検証

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3,
                                                    random_state=0, stratify=y_train) #stratify=階層化

In [ ]:
categorical_features = ['Embarked', 'Pclass', 'Sex']

## LightGBM

In [ ]:
import lightgbm as lgb #lgbのparamの書き方が変更されていることに注意
lgb_train = lgb.Dataset(X_train, y_train, 
                       categorical_feature=categorical_features)

lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, 
                       categorical_feature=categorical_features)
params = {
    'objective' : 'binary'
}

In [ ]:
model = lgb.train(params, lgb_train,
                 valid_sets=[lgb_train, lgb_eval],
                 num_boost_round=1000,
                 callbacks=[lgb.early_stopping(stopping_rounds=10,
                           verbose=True),
                           lgb.log_evaluation(10)])
                  
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm_holdout.csv', index=False)
sub.head(10)

[y_pred > 0.5]ならば[y == 1]、そうでなければ[y == 0]という離散値に変換している。\
TrueもしくはFalseが得られるので、astype(int)で「1もしくは0」に変換している。

## パラメーター調整

### 手動調整

In [ ]:
params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train,
                        categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                       categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  num_boost_round=1000,
                  callbacks=[lgb.early_stopping(stopping_rounds=10,
                                                verbose=True),
                             lgb.log_evaluation(10)])

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')

sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm_handtuning.csv', index=False)

sub.head()

### Optuna

In [ ]:
import optuna
from sklearn.metrics import log_loss

def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500), #255 - 500の間の整数
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }
    
    lgb_train = lgb.Dataset(X_train, y_train,
                           categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid,reference=lgb_train,
                           categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train,
                     valid_sets=[lgb_train, lgb_eval],
                     num_boost_round=1000,
                     callbacks=[lgb.early_stopping(stopping_rounds=10,
                                                  verbose=True),
                               lgb.log_evaluation(10)])
    
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

In [ ]:
params = { 
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': 0.05,
    'num_leaves': study.best_params['num_leaves']
}

lgb_train = lgb.Dataset(X_train, y_train,
                      categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                      categorical_feature=categorical_features)

model = lgb.train(params, lgb_train,
                 valid_sets=[lgb_train, lgb_eval],
                 num_boost_round=1000,
                 callbacks=[lgb.early_stopping(stopping_rounds=10,
                                              verbose=True),
                           lgb.log_evaluation(10)])

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm_optuna.csv', index=False)
sub.head()

## Cross Validation

In [ ]:
import numpy as np
import pandas as pd


train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'] = data['Sex'].replace({'male':0, 'female':1}).astype(int)
data['Embarked'] = data['Embarked'].fillna(('S'))
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'] = data['Fare'].fillna(np.mean(data['Fare']))
data['Age'] = data['Age'].fillna(data['Age'].median())
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
X_train.head()

In [ ]:
from sklearn.model_selection import KFold

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    lgb_train = lgb.Dataset(X_tr, y_tr,
                           categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                          categorical_feature=categorical_features)
    
    model = lgb.train(params, lgb_train,
                     valid_sets=[lgb_train, lgb_eval],
                     num_boost_round=1000,
                     callbacks=[lgb.early_stopping(stopping_rounds=10,
                                                  verbose=True),
                               lgb.log_evaluation(10)])
    
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration) #iteration=反復
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

In [ ]:
scores = [m.best_score['valid_1']['binary_logloss'] for m in models]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
len(y_preds)

In [ ]:
y_preds[0][:10]

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

In [ ]:
sub['Survived'] = y_sub
sub.to_csv('submission_lightgbm_skfold.csv', index=False)

sub.head()

## データセットの分割方法

In [ ]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    print(f'fold_id: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

最初にfをつけ、変数は{}で括ることで文字列の中に取り入れることができる。

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    print(f'fold_id: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

In [ ]:
from sklearn.model_selection import StratifiedKFold


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                            categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                           categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=1000,
                      callbacks=[lgb.early_stopping(stopping_rounds=10,
                                                    verbose=True),
                                 lgb.log_evaluation(10)])

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_skfold', index=False)
print(oof_train[:10])

scores = [m.best_score['valid_1']['binary_logloss'] for m in models]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
sub_lgbm_sk = pd.read_csv('/kaggle/working/submission_lightgbm_skfold.csv')
sub_lgbm_ho = pd.read_csv('/kaggle/working/submission_lightgbm_holdout.csv')
sub_rf = pd.read_csv('/kaggle/working/submission_randomforest.csv')

In [ ]:
df = pd.DataFrame({'sub_lgbm_sk': sub_lgbm_sk['Survived'].values,
                  'sub_lgbm_ho': sub_lgbm_ho['Survived'].values,
                  'sub_rf': sub_rf['Survived'].values})
df.head()

In [ ]:
# 相関関係を見る
df.corr()

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')
sub['Survived'] = sub_lgbm_sk['Survived'] + sub_lgbm_ho['Survived'] + sub_rf['Survived']
sub.head(10)

In [ ]:
sub['Survived'] = (sub['Survived'] >= 2).astype(int)
sub.to_csv('submission_lightgbm_ensemble.csv', index=False)
sub.head(10)